![Tinder](https://full-stack-assets.s3.eu-west-3.amazonaws.com/M03-EDA/Tinder-Symbole.png)

# Analyse du fichier retravaillé speed dating

## 📋 Objectif

**Comprendre les relations humaines** nous allons voir qu'est ce qui est important pour un match.

---

**les questions que nous essayerons de répondre sont les suivantes :**

- Est ce que les hommes et les femmes ont les meme attentes ?
- Pourquoi match t il ?
- y a t'il une importance du niveau social dans un match ?
- est ce que les attentes sont elle les meme selon leur classe d'age ?

In [181]:
# import des librairies 
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

# import de la charte graphique
from charte_graphique import *


In [182]:
df = pd.read_csv('../speed_dating_clean.csv')
df.head()

,id_unique,id_wave,genre,id_genre,vague,nb_personnes_vague,ordre_rencontre,id_partenaire,id_unique_partenaire,match,...,pref_intelligence,pref_fun,pref_ambition,pref_interets,auto_attractivite,auto_sincerite,auto_intelligence,auto_fun,auto_ambition,tranche_age
0,1,1.0,femme,1,1,10,4,1,11.0,non,...,20.0,15.0,15.0,15.0,6.0,8.0,8.0,8.0,7.0,18-24
1,1,1.0,femme,1,1,10,3,2,12.0,non,...,20.0,15.0,15.0,15.0,6.0,8.0,8.0,8.0,7.0,18-24
2,1,1.0,femme,1,1,10,10,3,13.0,oui,...,20.0,15.0,15.0,15.0,6.0,8.0,8.0,8.0,7.0,18-24
3,1,1.0,femme,1,1,10,5,4,14.0,oui,...,20.0,15.0,15.0,15.0,6.0,8.0,8.0,8.0,7.0,18-24
4,1,1.0,femme,1,1,10,7,5,15.0,oui,...,20.0,15.0,15.0,15.0,6.0,8.0,8.0,8.0,7.0,18-24


In [183]:
# recontrole rapide des données
print(f'il y a {df.shape[0]} lignes et {df.shape[1]} colonnes dans le dataset nettoyé.')
print(f'il y a {df.isnull().sum().sum()} valeurs manquantes dans le dataset nettoyé.')

il y a 7976 lignes et 81 colonnes dans le dataset nettoyé.
il y a 152 valeurs manquantes dans le dataset nettoyé.


## 1/ 👩‍❤️‍👨 Pourquoi venir et que recherchent les individus lors d'un speed dating ?

### analyse des individus combien de femmes et d'hommes

In [184]:
repartition_des_genres = df.groupby('id_unique')['genre'].first().value_counts()
repartition_des_genres

genre
homme    263
femme    260
Name: count, dtype: int64

In [245]:
df.columns.tolist()

['id_unique',
 'id_wave',
 'genre',
 'id_genre',
 'vague',
 'nb_personnes_vague',
 'ordre_rencontre',
 'id_partenaire',
 'id_unique_partenaire',
 'match',
 'age_partenaire',
 'race_partenaire',
 'decision_partenaire',
 'correlation_interets',
 'same_race',
 'deja_rencontre_partenaire',
 'attractivite_recue',
 'sincerite_recue',
 'intelligence_recue',
 'fun_recu',
 'ambition_recue',
 'interets_partages_recus',
 'appreciation_recue',
 'probabilite_oui_recue',
 'pref_partenaire_attractivite',
 'pref_partenaire_sincerite',
 'pref_partenaire_intelligence',
 'pref_partenaire_fun',
 'pref_partenaire_ambition',
 'pref_partenaire_interets',
 'decision',
 'attractivite_donnee',
 'sincerite_donnee',
 'intelligence_donnee',
 'fun_donne',
 'ambition_donnee',
 'interets_partages_donnes',
 'appreciation_donnee',
 'probabilite_oui_donnee',
 'deja_rencontre',
 'age',
 'domaine_etude',
 'domaine_code',
 'race',
 'importance_meme_race',
 'importance_meme_religion',
 'objectif_participation',
 'frequence_

In [185]:
# representation graphique du découpage homme femmes

fig = px.pie(
    names=repartition_des_genres.index,
    values=repartition_des_genres.values,
    title="Répartition des Genres dans le Dataset Speed Dating",
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig.show()

In [186]:
# Regroupement des âges en tranches
age_genre_counts = df.groupby(['tranche_age', 'genre'], observed=True)['id_unique'].nunique().unstack().fillna(0)
age_genre_counts


genre,femme,homme
tranche_age,,
18-24,98.0,82.0
25-34,155.0,177.0
35-44,6.0,4.0
45-59,1.0,0.0


### Pourquoi les gens viennent aux Speed dating ?


In [187]:
objectif = df.groupby(['objectif_participation', 'genre','id_unique'])['genre'].count()
objectif

objectif_participation                     genre  id_unique
Autre                                      femme  3            10
                                                  21           16
                                                  33           16
                                                  154          16
                                                  180          10
                                                               ..
Ça avait l'air d'être une soirée amusante  homme  545          22
                                                  548          22
                                                  549          22
                                                  551          22
                                                  552          22
Name: genre, Length: 523, dtype: int64

In [188]:
# Representation graphique des objetif de venirs au speed dating par type de genre
objectif = df.groupby(['objectif_participation','genre'])['id_unique'].nunique().reset_index(name='count')
# representation graphique du découpage par tranche d'age et par sexe
fig = px.bar(
    objectif,
    x='objectif_participation',
    y='count',
    color='genre',
    barmode='group',
    title="Répartition des objectifs de Venir au Speed Dating par Genre",
    labels={"count": "Nombre d'Individus", "objectif_participation": "Objectif", "genre": "Genre"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    xaxis=dict(
        tickangle=-45,
        tickfont=dict(size=9),
        tickmode='linear'
    ),
    margin=dict(b=120, l=60, r=40, t=80)
)
fig.show()

## 💔la premiere conclusion est : Les personnes ne viennent pas pour rencontrer l'amour mais soit pour s'amuser soit pour faire de nouvelles connaissance. il ne cible pas forcement la recherche d'un partenaire !! 💔

In [189]:
# representation graphique du découpage par tranche d'age et par sexe
fig = px.bar(
    age_genre_counts,
    barmode='group',
    title="Répartition des Genres par Tranche d'Âge",
    labels={"value": "Nombre d'Individus", "tranche_age": "Tranche d'Âge", "genre": "Genre"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig_layout = fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)
fig.show()

In [190]:
nb_match = df.groupby('genre')['match'].apply(lambda x: (x == 'oui').sum()) 
print(nb_match)
print('il y a bien une réciprocité dans les matchs entre hommes et femmes, il n y a pas de match seul !!')

genre
femme    669
homme    669
Name: match, dtype: int64
il y a bien une réciprocité dans les matchs entre hommes et femmes, il n y a pas de match seul !!


In [191]:
# sur quel categorie d'age y a t'il le plus de match ?
age_match_counts = df[df['match']=='oui'].groupby(['tranche_age', 'genre'], observed=True)['match'].count()
age_match_counts
# combien de match a total sur le nombre de recontres ?
total_matches = df[df['match']=='oui'].shape[0]
total_encounters = df.shape[0]
match_rate = (total_matches / total_encounters) * 100
print(f'Taux de match global : {match_rate:.2f}%')

Taux de match global : 16.78%


In [192]:
# representation graphique du découpage par tranche d'age et par sexe
fig= px.bar(
    age_match_counts.unstack().fillna(0),
    barmode='group',
    title="Nombre de Matchs par Tranche d'Âge et par Genre",
    labels={"value": "Nombre de Matchs", "tranche_age": "Tranche d'Âge", "genre": "Genre"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.show()

In [193]:
# analyse des préférences par genre en faisant la moyenne
preferences_by_genre = df.groupby('genre', observed=True).apply(lambda x: x[['pref_attractivite', 'pref_sincerite', 'pref_intelligence', 'pref_fun', 'pref_ambition', 'pref_interets']].mean())
preferences_genre = preferences_by_genre.reset_index()
preferences_genre = preferences_genre.melt(id_vars='genre', var_name='critere', value_name='moyenne')
preferences_genre

# representation graphique des préférences par genre
fig = px.bar(
    preferences_genre,
    x='critere',
    y='moyenne',
    color='genre',
    barmode='group',
    title="Préférences Moyennes par Critère et par Genre",
    labels={"moyenne": "Note Moyenne", "critere": "Critère", "genre": "Genre"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig.update_traces(texttemplate='%{y:.2f}', textposition='inside')
fig.show()

/var/folders/h5/tjbnq0312nnbccsycqjy_ftc0000gn/T/ipykernel_88936/3561260159.py:2: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# On voit que les femmes et les hommes recherchent :
    1- pour les hommes : l'attractivité
    2- pour les femmes : l'intelligence

est ce que cela differe t il en fonction du groupe d'age ?

In [194]:
# on va faire la moyenne des préférences par genre et par tranche d'age
preferences_by_genre_age = df.groupby(['genre','tranche_age'], observed=True).apply(lambda x: x[['pref_attractivite', 'pref_sincerite', 'pref_intelligence', 'pref_fun', 'pref_ambition', 'pref_interets']].mean())
preferences_genre_age = preferences_by_genre_age.reset_index()
preferences_genre_age = preferences_genre_age.melt(id_vars=['genre', 'tranche_age'], var_name='critere', value_name='moyenne')
preferences_genre_age
# representation graphique des préférences par tranche d'age et un graphique par genre

#1 les femmes

fig_femmes = px.bar(
    preferences_genre_age[preferences_genre_age['genre'] == 'femme'],
    x='critere',
    y='moyenne',
    color='tranche_age',
    barmode='group',
    title="Préférences Moyennes des Femmes par Critère et Tranche d'Âge",
    labels={"moyenne": "Note Moyenne", "critere": "Critère", "tranche_age": "Tranche d'Âge"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_femmes.update_traces(texttemplate='%{y:.2f}', textposition='inside')
fig_femmes.show()

#2 les hommes
fig_hommes = px.bar(
    preferences_genre_age[preferences_genre_age['genre'] == 'homme'],
    x='critere',
    y='moyenne',
    color='tranche_age',
    barmode='group',
    title="Préférences Moyennes des Hommes par Critère et Tranche d'Âge",
    labels={"moyenne": "Note Moyenne", "critere": "Critère", "tranche_age": "Tranche d'Âge"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_hommes.update_traces(texttemplate='%{y:.2f}', textposition='inside')
fig_hommes.show()




/var/folders/h5/tjbnq0312nnbccsycqjy_ftc0000gn/T/ipykernel_88936/586863641.py:2: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



on peut constater que les attentes sont a peu pres identiques selon l'age.
Attention, le fun pour les hommes est important pour les hommes de 35 - 44 ans

nous allons voir si le match correspond a leur attente enfonction des notes qu'ils ont perçus.

In [195]:
# nous allons comparer les match en fonction des préférences exprimées et des notes reçues

#filtrer sur les match oui
df_matchs = df[df['match']=='oui']
# il va falloir regrouper les notes reçues par critère et par genre
notes_by_genre = df_matchs.groupby('genre', observed=True).apply(lambda x: x[['attractivite_recue', 'sincerite_recue', 'intelligence_recue', 'fun_recu', 'ambition_recue', 'interets_partages_recus']].mean())
notes_genre_recu= notes_by_genre.reset_index()
notes_genre_recu = notes_genre_recu.melt(id_vars='genre', var_name='critere', value_name='moyenne')
notes_genre_recu
# representation graphique des notes reçues par critère et par genre
fig_notes_recue = px.line_polar(
    notes_genre_recu,
    r='moyenne',
    theta='critere',
    color='genre',
    line_close=True,
    title="Notes Moyennes Reçues par Critère et par Genre (Matchs Oui)",
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_notes_recue.show()

/var/folders/h5/tjbnq0312nnbccsycqjy_ftc0000gn/T/ipykernel_88936/3021251383.py:6: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



on peut constater qu'il s'agit d'une notation sur 10 avec qq chose d'homogene, mais que les hommes n'on pas forcement mis en avant l'attractivité en premier plan dans leur notation on s'aperçoit qu'il s'agit de l'intelligence. et les femmes ont bien note que les hommes intelligents on un match.

pour vérifier s'il n y a pas d'erreur sur les notes on va faire sur les données pour valider la réciprocité.

In [196]:
# reprendre la même chose pour les notes données

#filtrer sur les match oui
df_matchs = df[df['match']=='oui']
# il va falloir regrouper les notes reçues par critère et par genre
notes_by_genre = df_matchs.groupby('genre', observed=True).apply(lambda x: x[['attractivite_donnee', 'sincerite_donnee', 'intelligence_donnee', 'fun_donne', 'ambition_donnee', 'interets_partages_donnes']].mean())
notes_genre = notes_by_genre.reset_index()
notes_genre = notes_genre.melt(id_vars='genre', var_name='critere', value_name='moyenne')
notes_genre
# representation graphique des notes données par critère et par genre
fig_notes = px.line_polar(
    notes_genre,
    r='moyenne',
    theta='critere',
    color='genre',
    line_close=True,
    title="Notes Moyennes Données par Critère et par Genre (Matchs Oui)",
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_notes.show()

/var/folders/h5/tjbnq0312nnbccsycqjy_ftc0000gn/T/ipykernel_88936/3000568381.py:6: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



on va regarder egalement par rapport a son auto evaluation.

In [197]:
# on va comparer egalement par rapport a son auto evaluation.

# il va falloir regrouper les notes reçues par critère et par genre
auto_eval_by_genre = df.groupby('genre', observed=True).apply(lambda x : x[['auto_attractivite', 'auto_sincerite', 'auto_intelligence', 'auto_fun', 'auto_ambition']].mean())
auto_eval_genre = auto_eval_by_genre.reset_index()
auto_eval_genre = auto_eval_genre.melt(id_vars='genre', var_name='critere', value_name='moyenne')
auto_eval_genre

# representation graphique des notes auto évaluées par critère et par genre
fig_auto_notes = px.line_polar(
    auto_eval_genre,
    r='moyenne',
    theta='critere',
    color='genre',
    line_close=True,
    title="Notes Moyennes Auto-Évaluées par Critère et par Genre",
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_auto_notes.show()

/var/folders/h5/tjbnq0312nnbccsycqjy_ftc0000gn/T/ipykernel_88936/4008890518.py:4: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# Comment nous voyons nous ? 

In [198]:
#  on va faire une analyse croisée entre l'auto evaluation et les notes recues
notes_genre_recu

,genre,critere,moyenne
0,femme,attractivite_recue,7.528401
1,homme,attractivite_recue,7.119581
2,femme,sincerite_recue,7.784753
3,homme,sincerite_recue,7.796712
4,femme,intelligence_recue,7.812407
5,homme,intelligence_recue,8.053812
6,femme,fun_recu,7.542601
7,homme,fun_recu,7.621824
8,femme,ambition_recue,7.172646
9,homme,ambition_recue,7.448430


In [199]:
auto_eval_genre

,genre,critere,moyenne
0,femme,auto_attractivite,7.212136
1,homme,auto_attractivite,6.952106
2,femme,auto_sincerite,8.409980
3,homme,auto_sincerite,8.136158
4,femme,auto_intelligence,8.302909
5,homme,auto_intelligence,8.485958
6,femme,auto_fun,7.879639
7,homme,auto_fun,7.514794
8,femme,auto_ambition,7.610080
9,homme,auto_ambition,7.574975


on va renommer les colonnes pour faire le merge. et ajouter une colonne type

In [200]:
#autoeval renommer les criteres
renamed_auto_eval_genre = auto_eval_genre.replace({
    'auto_attractivite': 'attractivite',
    'auto_sincerite': 'sincerite',
    'auto_intelligence': 'intelligence',
    'auto_fun': 'fun',
    'auto_ambition': 'ambition'
})
# ajout de la colonne type pour autoeval
renamed_auto_eval_genre['type'] = 'auto_evaluation'
renamed_auto_eval_genre

,genre,critere,moyenne,type
0,femme,attractivite,7.212136,auto_evaluation
1,homme,attractivite,6.952106,auto_evaluation
2,femme,sincerite,8.409980,auto_evaluation
3,homme,sincerite,8.136158,auto_evaluation
4,femme,intelligence,8.302909,auto_evaluation
5,homme,intelligence,8.485958,auto_evaluation
6,femme,fun,7.879639,auto_evaluation
7,homme,fun,7.514794,auto_evaluation
8,femme,ambition,7.610080,auto_evaluation
9,homme,ambition,7.574975,auto_evaluation


In [201]:
#note_recues renommer les criteres
renamed_note_recu= notes_genre_recu.replace({
    'attractivite_recue': 'attractivite',
    'sincerite_recue': 'sincerite',
    'intelligence_recue': 'intelligence',
    'fun_recu': 'fun',
    'ambition_recue': 'ambition'
})
# ajout de la colonne type pour note recues
renamed_note_recu['type'] = 'note_recue'

# on retire le critere interets partagés car il n'est pas dans l'auto evaluation
renamed_note_recu = renamed_note_recu[renamed_note_recu['critere'] != 'interets_partages_recus']
renamed_note_recu

,genre,critere,moyenne,type
0,femme,attractivite,7.528401,note_recue
1,homme,attractivite,7.119581,note_recue
2,femme,sincerite,7.784753,note_recue
3,homme,sincerite,7.796712,note_recue
4,femme,intelligence,7.812407,note_recue
5,homme,intelligence,8.053812,note_recue
6,femme,fun,7.542601,note_recue
7,homme,fun,7.621824,note_recue
8,femme,ambition,7.172646,note_recue
9,homme,ambition,7.448430,note_recue


In [202]:
# representation graphique de l'analyse comparative
merged=pd.concat([renamed_auto_eval_genre, renamed_note_recu], ignore_index=True)
merged

fig_comparative_femme = px.line_polar(
    merged[merged['genre']=='femme'],
    r='moyenne',
    theta='critere',
    color='type',
    line_close=True,
    title="Comparaison entre Auto-Évaluation et Notes Reçues par Critère selon les Femmes",
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_comparative_femme.show()

fig_comparative_homme = px.line_polar(
    merged[merged['genre']=='homme'],
    r='moyenne',
    theta='critere',
    color='type',
    line_close=True,
    title="Comparaison entre Auto-Évaluation et Notes Reçues par Critère selon les Hommes",
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_comparative_homme.show()

In [ ]:
# est ce que le niveau social a une influence dans les match ?
social_match_counts = df[df['match']=='oui'].groupby(['genre', 'carriere_code'], observed=True)['id_unique'].nunique().reset_index(name='count')
social_match_counts

fig_social = px.bar(
    social_match_counts.sort_values(by='count', ascending=False),
    x='carriere_code',
    y='count',
    barmode='group',
    color='genre',
    title="Nombre de Matchs par Catégorie Sociale et par Genre",
    labels={"count": "Nombre de Matchs", "carriere_code": "Catégorie Sociale", "genre": "Genre"},
    template="nicolas_herve",
    color_discrete_sequence=COLOR_PALETTE
)
fig_social.update_traces(texttemplate='%{y}', textposition='inside')
fig_social.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    xaxis=dict(
        tickangle=-45,
        tickfont=dict(size=10
                      ),
        tickmode='linear'
    ),
    margin=dict(b=120, l=60, r=40, t=80)
)
fig_social.show()

Relation entre les interets hobbies et un match

In [241]:
# tableau de correlation entre les interets hobbies et un match
interests_columns = [
'interet_sport',
'interet_sport_tv',
'interet_exercice',
'interet_restaurant',
'interet_musees',
'interet_art',
'interet_randonnee',
'interet_jeux',
'interet_boite_nuit',
'interet_lecture',
'interet_tv',
'interet_theatre',
'interet_cinema',
'interet_concerts',
'interet_musique',
'interet_shopping',
'interet_yoga'
]
interests_match_corr = df[interests_columns + ['match']].copy()
# convertir la colonne match en binaire
interests_match_corr['match'] = interests_match_corr['match'].map({'oui': 1, 'non': 0})
correlation_matrix = interests_match_corr.corr()
correlation_with_match = correlation_matrix['match'].drop('match').sort_values(ascending=False)
correlation_matrix


#matrice de correlation entre les interets et le match
fig_corr = px.imshow(
    correlation_matrix,
    labels=dict(x="Variables", y="Variables", color="Corrélation"),
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    color_continuous_scale='RdBu',
    title="Matrice de Corrélation entre les Intérêts et le Match",
    template="nicolas_herve"
)
fig_corr.update_xaxes(tickangle=45, tickfont=dict(size=10))
fig_corr.show()

on va analyser les importances ethniques


In [244]:
# correlation entre les importances ethniques et un match
ethnique= df[['importance_ethnique', 'match']].copy()
# convertir la colonne match en binaire
ethnique['match'] = ethnique['match'].map({'oui': 1, 'non': 0})
ethnique_corr = ethnique.corr()
ethnique_corr

KeyError: "['importance_ethnique'] not in index"